In [ ]:
import os
import pandas as pd

# === 1. Set Case Folder ===
base_path = "/home/jovyan/my_android_logs/CASE_FILES_raw_logs"
case_name = input("📁 Enter the case folder name: ").strip()
case_folder = os.path.join(base_path, case_name)

# === 2. File Paths ===
port_file = os.path.join(case_folder, f"{case_name}_PORT_SCAN.csv")
app_file = os.path.join(case_folder, f"{case_name}_IP_FINAL_REPORT.csv")
output_file = os.path.join(case_folder, f"{case_name}_PORT_BEHAVIOR_REPORT.csv")

# === 3. Check if port scan file exists ===
if not os.path.exists(port_file):
    print(f"Port scan file not found: {port_file}")
    print("Please run the port scan first.")
else:
    # === 4. Load Data ===
    df_ports = pd.read_csv(port_file)
    df_apps = pd.read_csv(app_file)

    # === 5. Normalize IP Columns ===
    df_ports["ip"] = df_ports["ip"].astype(str).str.strip()
    df_apps["ip"] = df_apps["ip"].astype(str).str.strip()

    # === 6. Fix column if needed
    if "process_name" in df_apps.columns and "app_name" not in df_apps.columns:
        df_apps.rename(columns={"process_name": "app_name"}, inplace=True)

    # === 7. Merge on IP (preserve all details from both files)
    merged = pd.merge(df_ports, df_apps, on="ip", how="left")

    # === 8. Add Risk Level (based on 'tag' column if it exists)
    if "tag" in merged.columns:
        merged["risk_level"] = merged["tag"].str.upper().fillna("LOW")
    else:
        merged["risk_level"] = "LOW"

    # === 9. Save Final Output
    merged.to_csv(output_file, index=False)
    print(f"\n✅ Final report saved to: {output_file}")

    # === 10. Show All Columns (First Few Rows)
    display(merged.head())  # show all merged columns, not just a subset
